In [114]:
import pandas as pd
import numpy as np
import pandas_datareader as pdr
import matplotlib.pyplot as plt

#Currencies
peso_dollar = pdr.get_data_fred("DEXMXUS", "1975-01-01", "2019-12-31")
peso_dollar = peso_dollar.squeeze()
HK_dollar = pdr.get_data_fred("DEXHKUS", "1975-01-01", "2019-12-31")
dollar_pound = pdr.get_data_fred("DEXUSUK", "1975-01-01", "2019-12-31")
dollar_pound = dollar_pound.squeeze()

#Create / currency
HK_GBP = HK_dollar
HK_GBP["DEXHKUK"] = HK_dollar["DEXHKUS"] / dollar_pound
HK_GBP = HK_GBP.drop(columns=["DEXHKUS"])
HK_GBP = HK_GBP.squeeze()


#indexes
KS11 = pdr.get_data_yahoo('^KS11', start='1997-07-01', end='2019-12-31')
KS11 = KS11.drop(columns=["High", "Low", "Open", "Volume", "Adj Close"])
KS11 = KS11.squeeze()
BVSP = pdr.get_data_yahoo('^BVSP', start='1993-04-23', end='2019-12-31')
BVSP = BVSP.drop(columns=["High", "Low", "Open", "Volume", "Adj Close"])
BVSP = BVSP.squeeze()
DAX = pdr.get_data_yahoo('^GDAXI', start='1987-12-30', end='2019-12-31')
DAX = DAX.drop(columns=["High", "Low", "Open", "Volume", "Adj Close"])
DAX = DAX.squeeze()
VBTLX = pdr.get_data_yahoo('VBTLX', start='2001-11-12', end='2019-12-31')
VBTLX = VBTLX.drop(columns=["High", "Low", "Open", "Volume", "Adj Close"])
VBTLX = VBTLX.squeeze()

#Factors
ff_factors = pdr.get_data_famafrench("F-F_Research_Data_Factors_daily", start="1920", end="2019-12-31")[0]
ff_factors_mom = pdr.get_data_famafrench("F-F_Momentum_Factor_daily", start="1920", end="2019-12-31")[0]
# ff_factors = ff_factors.join(ff_factors_mom).dropna()
ff_factors = pd.concat([ff_factors,ff_factors_mom], axis=1)
ff_factors = ff_factors.dropna()
ff_factors.columns = [col.strip(' ') for col in ff_factors.columns]
ff_factors_price = (1+(ff_factors/100)).cumprod()

In [86]:
import numpy as np
import pandas as pd

def summary_statistics(prices):

    d_returns = prices.pct_change().dropna()
    w_returns = prices.resample("W-FRI").last().pct_change().dropna()
    m_returns = prices.resample("M").last().pct_change().dropna()
    q_returns = prices.resample("Q").last().pct_change().dropna()
    y_returns = prices.resample("A").last().pct_change().dropna()
    # Convert prices to returns

    # Enclosed function to find mean, std, skew and kurt: gives annualised stats
    def process(returns, n):
        returns_mu = returns.mean()                         # mean returns per time period
        annual_mu = ((1 + returns_mu) ** n) -1                          # annualised mean returns

        returns_err = returns - returns_mu       
        returns_var = (returns_err ** 2).mean()             # average squared return ( 2nd moment)
        annual_var = returns_var * n                        # annual rescaled variance
        annual_std = np.sqrt(annual_var)                    # annual rescaled std deviation

        returns_mom3 = (returns_err ** 3).mean()
        returns_mom4 = (returns_err ** 4).mean()            # Third and Fourth moments


        annual_skew = (returns_mom3 / returns_var ** (3/2)) #/ np.sqrt(n)  # rescaled Skew and Kurtosis
        annual_kurt = (returns_mom4 / returns_var ** (4/2)) #/ n

        returns_stats = pd.Series([annual_mu, annual_std, annual_skew, annual_kurt], index = ["mean", "std", "skew", "kurt"])
        
        return returns_stats
         
    # Feed in daily, weekly, monthly and quarterly data
    d_stats = process(d_returns, n = 252).rename("Daily")
    w_stats = process(w_returns, n = 52 ).rename("Weekly")
    m_stats = process(m_returns, n = 12 ).rename("Monthly")
    q_stats = process(q_returns, n = 4  ).rename("Quarterly")
    y_stats = process(y_returns, n = 1  ).rename("Annual")

    # Concatenate the series into a dataframe
    stats = pd.concat([d_stats, w_stats, m_stats, q_stats, y_stats], axis=1)

    return stats

In [113]:
print(summary_statistics(HK_GBP).to_latex())

\begin{tabular}{lrrrrr}
\toprule
{} &      Daily &    Weekly &   Monthly &  Quarterly &    Annual \\
\midrule
mean &   0.030951 &  0.032066 &  0.031204 &   0.030546 &  0.027004 \\
std  &   0.106572 &  0.109615 &  0.109426 &   0.117676 &  0.138505 \\
skew &   0.616992 &  0.670850 &  0.417659 &   0.795916 &  0.756487 \\
kurt &  11.902801 &  7.721188 &  4.864608 &   4.914809 &  3.263226 \\
\bottomrule
\end{tabular}



In [65]:
round(summary_statistics(peso_dollar),4)

,Daily,Weekly,Monthly,Quarterly,Annual
mean,0.0793,0.0785,0.0816,0.0831,0.0834
std,0.1445,0.1345,0.1441,0.1574,0.1710
skew,5.1194,6.1748,4.5461,2.9866,1.8030
kurt,148.9498,106.7444,45.5237,15.4730,5.7393


In [75]:
np.round(summary_statistics(KS11),4)


,Daily,Weekly,Monthly,Quarterly,Annual
mean,0.0892,0.0859,0.0900,0.1080,0.1285
std,0.2720,0.2699,0.2761,0.3229,0.3046
skew,-0.0253,-0.1441,0.8465,1.1980,0.1047
kurt,8.8864,8.0447,8.4423,9.0319,3.0305


In [87]:
np.round(summary_statistics(VBTLX),4)

,Daily,Weekly,Monthly,Quarterly,Annual
mean,0.0038,0.0049,0.0047,0.0053,0.0051
std,0.0373,0.0358,0.0343,0.0325,0.0247
skew,-0.1019,-0.5142,-0.2546,-0.2109,-0.0504
kurt,4.4780,4.1618,4.4164,2.4039,2.8464


In [76]:
np.round(summary_statistics(DAX),4)

,Daily,Weekly,Monthly,Quarterly,Annual
mean,0.1103,0.1083,0.1067,0.1126,0.1115
std,0.2198,0.2117,0.2026,0.2260,0.2299
skew,-0.0816,-0.3506,-0.5184,-0.6308,-0.6526
kurt,8.7932,6.8651,5.0618,4.6575,2.7851


In [77]:
np.round(summary_statistics(BVSP),4)

,Daily,Weekly,Monthly,Quarterly,Annual
mean,0.4704,0.4577,0.4881,0.5663,0.6072
std,0.3553,0.3515,0.4369,0.8282,2.0417
skew,0.9501,0.6316,2.2875,4.4751,4.4806
kurt,17.7566,7.5002,15.0970,27.7436,22.0358


In [72]:
np.round(summary_statistics(ff_factors_price["Mom"]),4)

,Daily,Weekly,Monthly,Quarterly,Annual
mean,0.0691,0.0744,0.0771,0.0772,0.0770
std,0.1183,0.1373,0.1519,0.1507,0.1538
skew,-1.5947,-1.3896,-0.9152,-0.9939,-0.7214
kurt,29.8645,15.5112,11.3743,9.2948,5.3789


In [98]:
np.round(summary_statistics(ff_factors_price["SMB"]),4)

,Daily,Weekly,Monthly,Quarterly,Annual
mean,0.0116,0.0118,0.0132,0.0141,0.0144
std,0.0925,0.0912,0.1066,0.1159,0.1169
skew,-0.7553,0.0826,1.1168,1.2164,0.2262
kurt,25.6500,13.1245,13.6022,10.2252,4.1801


In [74]:
np.round(summary_statistics(ff_factors_price["HML"]),4)

,Daily,Weekly,Monthly,Quarterly,Annual
mean,0.0397,0.0421,0.0436,0.0459,0.0450
std,0.0930,0.1025,0.1149,0.1352,0.1303
skew,0.7319,1.1020,1.2923,1.3357,0.3532
kurt,18.7482,16.1467,12.3441,11.9692,3.4701


In [109]:
np.round(summary_statistics(ff_factors_price["Mkt-RF"]),4)

,Daily,Weekly,Monthly,Quarterly,Annual
mean,0.0772,0.0804,0.0828,0.0881,0.0840
std,0.1687,0.1716,0.1845,0.2202,0.1975
skew,-0.1205,-0.3581,0.1932,1.8223,-0.2983
kurt,19.6013,9.1428,10.9329,19.8339,2.9819


In [187]:
KS11_2019 = summary_statistics(KS11["31/12/2009":"31/12/2019"])
KS11_2019 = KS11_2019.drop(columns="Annual")
KS11_2019 = np.round(KS11_2019,4)
KS11_2019

,Daily,Weekly,Monthly,Quarterly
mean,0.0383,0.0367,0.0412,0.0342
std,0.1479,0.1420,0.1299,0.1165
skew,-0.3750,-0.6276,-0.5454,-0.5877
kurt,6.8910,5.0810,4.3409,3.6521


from scipy.special import gammaln
from scipy.optimize import minimize

def students_dof(prices):
    def std_t_loglik(nu, x):
        # These are fixed for now
        mu = x.mean()
        sigma2 = x.var()
        sigma = np.sqrt(sigma2)

        a = gammaln((nu + 1) / 2)
        b = gammaln(nu / 2)
        c = np.sqrt(np.pi * (nu-2))
        d = ((nu + 1) / 2)
        e = (x - mu) **2
        f = sigma2 * (nu - 2)

        loglik = a - b - np.log(c) - np.log(sigma) - d * np.log(1 + e / f)
        return -(loglik.sum())
    
    d_returns = prices.pct_change().dropna()
    w_returns = prices.resample("W-FRI").last().pct_change().dropna()
    m_returns = prices.resample("M").last().pct_change().dropna()
    q_returns = prices.resample("Q").last().pct_change().dropna()

    starting_val = np.array([3]) 
    opt = minimize(std_t_loglik, starting_val, args=(w_returns), bounds=[(2.01, 100)], options={"disp": True})
    mle = opt.x[0]   
    return mle

In [145]:
def dof(prices):
    
    def calc_kurt(series):
        mu = series.mean()
        err = series - mu
        var = (err**2).mean()
        mom4 = (err**4).mean()
        kurt = mom4 / var ** (4/2)
        return kurt
    
    d_k = calc_kurt(prices.pct_change())
    w_k = calc_kurt(prices.resample("W-FRI").last().pct_change())
    m_k = calc_kurt(prices.resample("M").last().pct_change())
    q_k = calc_kurt(prices.resample("Q").last().pct_change())

    def df(k):
        a = k - 2
        b = k - 4
        df = 3 * (a/b)
        if k > 4:
            df=df
        else:
            df = float("inf")
        return df
    
    df_d = df(d_k)
    df_w = df(w_k)
    df_m = df(m_k)
    df_q = df(q_k)

    freedom = pd.Series([df_d, df_w, df_m, df_q], index = ["Daily", "Weekly", "Monthly", "Quarterly"])
    return freedom




In [159]:
df_DAX = dof(DAX)

In [160]:
df_BVSP = dof(BVSP)

In [170]:
df_HK = dof(HK_GBP)

In [169]:
df_KS11 = dof(KS11)

In [161]:
df_VBTLX = dof(VBTLX)

In [163]:
df_peso = dof(peso_dollar)

In [164]:
df_mom = dof(ff_factors_price["Mom"])

In [165]:
df_hml = dof(ff_factors_price["HML"])

In [166]:
df_smb = dof(ff_factors_price["SMB"])

In [168]:
df_mkt= dof(ff_factors_price["Mkt-RF"])

In [181]:
table = pd.concat([df_HK, df_peso, df_DAX, df_BVSP, df_KS11, df_VBTLX, df_mom, df_hml, df_smb, df_mkt], axis=1)
table.columns = ["HK/GBP", "MXN/USD", "DAX", "BSVP", "KS11", "VBTLX", "Mom", "HML", "SMB", "VWM"]
table

,HK/GBP,MXN/USD,DAX,BSVP,KS11,VBTLX,Mom,HML,SMB,VWM
Daily,3.759224,3.041394,4.251780,3.436155,4.227902,15.551533,3.231978,3.406828,3.277136,3.384582
Weekly,4.612388,3.058397,5.094197,4.714195,4.483435,40.074300,3.521230,3.493961,3.657573,4.166672
Monthly,9.939558,3.144496,8.651005,3.540684,4.350651,17.410601,3.813639,3.719072,3.624858,3.865438
Quarterly,9.558748,3.522967,12.125425,3.252700,4.192384,inf,4.133186,3.752896,3.963831,3.378935
